In [1]:
from seeq import spy
import pandas as pd
import numpy as np

In [2]:
# Log into Seeq Server if you're not using Seeq Data Lab:
spy.login(url='http://localhost:34216', credentials_file='../../credentials.key', force=False)

In [3]:
from seeq.spy.assets import Asset

first_area = {
    '1': 'Area A',
    '2': 'Area D'
}

class Area(Asset):
    @Asset.Display()
    def Detail(self, metadata, analysis):
        worksheet = analysis.worksheet('Area ' + self.definition['Name'])

        workstep = worksheet.workstep()
        workstep.display_items = metadata[metadata['Name'] == 'Temperature']
        workstep.view = 'Trend'

        return workstep

class Cooling_Tower(Asset):
    @Asset.Display()
    def Incident_Display(self, metadata, analysis):
        worksheet = analysis.worksheet('Cooling Tower ' + self.definition['Name'] + ' Overview')

        workstep = worksheet.workstep()
        workstep.data['version']=35
        workstep.data['state']['stores']['sqWorksheetStore']['viewKey'] = 'TABLE'
        workstep.data['state']['stores'].update(sqTableBuilderStore={})
        workstep.data['state']['stores']['sqTableBuilderStore'].update(assetId={})
        workstep.data['state']['stores']['sqTableBuilderStore']['assetId'].update(simple=metadata[metadata['Cooling Tower'] == self.definition['Asset']]['Cooling Tower ID'].iloc[0])
        workstep.data['state']['stores']['sqTableBuilderStore'].update(mode='simple')
        workstep.data['state']['stores']['sqTableBuilderStore'].update(columns={'simple': [{
            'accessor': 'assets',
            'headerTextAlign': 'center',
            'headerTextStyle': [ 'bold' ],
            'hideSort': True,
            'key': 'asset',
            'shortTitle': 'ASSET_HEADER',
            'style': 'assets',
            'title': 'ASSET_HEADER'
        },{
            'columnSuffix': 'Average',
            'headerTextAlign': 'center',
            'headerTextStyle': [ 'bold' ],
            'invalidsFirst': True,
            'key': "statistics.average",
            'shortTitle': 'STATISTICS.AVERAGE.SHORT',
            'stat': 'average()',
            'style': 'decimal',
            'title': 'STATISTICS.AVERAGE.LONG'
        }]})

        workstep.display_items = metadata[metadata['Asset'] == first_area[self.definition['Asset']]]

        return workstep

    @Asset.Component()
    def Areas(self, metadata):
        return self.build_components(Area, metadata, column_name='Area')

    @Asset.DateRange()
    def Last_Two_Months(self, metadata):
        return {
            'Start': '2019-11-01',
            'End': '2020-01-01'
        }

    @Asset.DateRange()
    def Last_Week(self, metadata):
        return {
            'Start': '2020-01-01',
            'End': '2020-01-08'
        }

    @Asset.Document()
    def Topic_Document(self, metadata, topic):
        document = topic.document('Cooling Tower ' + self.definition['Name'])
        document.render_template(filename='Cooling_Tower_Health.html', asset=self)

class Region(Asset):
    @Asset.Component()
    def Cooling_Towers(self, metadata):
        return self.build_components(Cooling_Tower, metadata, column_name='Cooling Tower')


In [4]:
search_df = spy.search({
    'Path': 'Example >> Cooling Tower ? >> /Area [A-K]/',
    'Type': 'Signal',
    'Name': 'Temperature'})

search_df

,Path,Type,Name,Time,Count,Result
0,Example >> Cooling Tower ? >> /Area [A-K]/,Signal,Temperature,00:00:00.22,0,Success


,ID,Path,Asset,Name,Description,Type,Value Unit Of Measure,Datasource Name,Archived
0,94478EFE-4273-4DA7-94F6-76252928D863,Example >> Cooling Tower 1,Area A,Temperature,NaN,StoredSignal,°F,Example Data,False
1,7F26820B-2538-447C-B17F-CD176D0A08EA,Example >> Cooling Tower 1,Area B,Temperature,NaN,StoredSignal,°F,Example Data,False
2,318EE867-6C9D-42BA-887E-86DB2E8AA162,Example >> Cooling Tower 1,Area C,Temperature,NaN,StoredSignal,°F,Example Data,False
3,154602B6-694A-43C9-A946-8401109B221E,Example >> Cooling Tower 1,Area G,Temperature,NaN,StoredSignal,°F,Example Data,False
4,DB50958E-3E6B-4DD3-96A9-57B3D8ECB95C,Example >> Cooling Tower 1,Area H,Temperature,NaN,StoredSignal,°F,Example Data,False
5,F029E5EA-E42F-4FA9-8406-AB186249FCD7,Example >> Cooling Tower 1,Area I,Temperature,NaN,StoredSignal,°F,Example Data,False
6,18E394C7-CF73-4409-8446-216865AEA9E0,Example >> Cooling Tower 1,Area J,Temperature,NaN,StoredSignal,°F,Example Data,False
7,41D9BEC4-7CF7-415D-AE0C-F9367646E545,Example >> Cooling Tower 1,Area K,Temperature,NaN,StoredSignal,°F,Example Data,False
8,8F5CAA7E-A9E8-4A9E-8968-4AD42D6D97EF,Example >> Cooling Tower 2,Area D,Temperature,NaN,StoredSignal,°F,Example Data,False
9,3C513530-DDF9-4A9F-8FB6-34AEC4879089,Example >> Cooling Tower 2,Area E,Temperature,NaN,StoredSignal,°F,Example Data,False


In [5]:
cooling_tower_df = spy.search({
    'Path': 'Example',
    'Name': 'Cooling Tower ?'})

cooling_tower_df['Name'] = cooling_tower_df['Name'].str.extract(r'Cooling Tower (\d)')
cooling_tower_df

,Path,Name,Time,Count,Result
0,Example,Cooling Tower ?,00:00:00.02,2,Success


,ID,Path,Asset,Name,Description,Type,Value Unit Of Measure,Datasource Name,Archived
0,AD91D929-E7F3-4ED1-805B-4F5573D483E4,NaN,Example,1,NaN,Asset,NaN,Example Data,False
1,29BE8A62-3F11-4EDD-BB85-2C7EC7DB332D,NaN,Example,2,NaN,Asset,NaN,Example Data,False


In [6]:
metadata_df = search_df.copy()
metadata_df['Cooling Tower'] = metadata_df['Path'].str.extract(r'Cooling Tower (\d)')
metadata_df['Area'] = metadata_df['Asset'].str.extract(r'Area (\w)')
metadata_df['Build Path'] = 'Cooling Tower Health'
metadata_df['Build Asset'] = 'North America'

def find_cooling_tower_id(cooling_tower):
    return cooling_tower_df[cooling_tower_df['Name'] == cooling_tower].values[0][0]
metadata_df['Cooling Tower ID'] = metadata_df['Cooling Tower'].apply(find_cooling_tower_id)
0
metadata_df

,ID,Path,Asset,Name,Description,Type,Value Unit Of Measure,Datasource Name,Archived,Cooling Tower,Area,Build Path,Build Asset,Cooling Tower ID
0,94478EFE-4273-4DA7-94F6-76252928D863,Example >> Cooling Tower 1,Area A,Temperature,NaN,StoredSignal,°F,Example Data,False,1,A,Cooling Tower Health,North America,AD91D929-E7F3-4ED1-805B-4F5573D483E4
1,7F26820B-2538-447C-B17F-CD176D0A08EA,Example >> Cooling Tower 1,Area B,Temperature,NaN,StoredSignal,°F,Example Data,False,1,B,Cooling Tower Health,North America,AD91D929-E7F3-4ED1-805B-4F5573D483E4
2,318EE867-6C9D-42BA-887E-86DB2E8AA162,Example >> Cooling Tower 1,Area C,Temperature,NaN,StoredSignal,°F,Example Data,False,1,C,Cooling Tower Health,North America,AD91D929-E7F3-4ED1-805B-4F5573D483E4
3,154602B6-694A-43C9-A946-8401109B221E,Example >> Cooling Tower 1,Area G,Temperature,NaN,StoredSignal,°F,Example Data,False,1,G,Cooling Tower Health,North America,AD91D929-E7F3-4ED1-805B-4F5573D483E4
4,DB50958E-3E6B-4DD3-96A9-57B3D8ECB95C,Example >> Cooling Tower 1,Area H,Temperature,NaN,StoredSignal,°F,Example Data,False,1,H,Cooling Tower Health,North America,AD91D929-E7F3-4ED1-805B-4F5573D483E4
5,F029E5EA-E42F-4FA9-8406-AB186249FCD7,Example >> Cooling Tower 1,Area I,Temperature,NaN,StoredSignal,°F,Example Data,False,1,I,Cooling Tower Health,North America,AD91D929-E7F3-4ED1-805B-4F5573D483E4
6,18E394C7-CF73-4409-8446-216865AEA9E0,Example >> Cooling Tower 1,Area J,Temperature,NaN,StoredSignal,°F,Example Data,False,1,J,Cooling Tower Health,North America,AD91D929-E7F3-4ED1-805B-4F5573D483E4
7,41D9BEC4-7CF7-415D-AE0C-F9367646E545,Example >> Cooling Tower 1,Area K,Temperature,NaN,StoredSignal,°F,Example Data,False,1,K,Cooling Tower Health,North America,AD91D929-E7F3-4ED1-805B-4F5573D483E4
8,8F5CAA7E-A9E8-4A9E-8968-4AD42D6D97EF,Example >> Cooling Tower 2,Area D,Temperature,NaN,StoredSignal,°F,Example Data,False,2,D,Cooling Tower Health,North America,29BE8A62-3F11-4EDD-BB85-2C7EC7DB332D
9,3C513530-DDF9-4A9F-8FB6-34AEC4879089,Example >> Cooling Tower 2,Area E,Temperature,NaN,StoredSignal,°F,Example Data,False,2,E,Cooling Tower Health,North America,29BE8A62-3F11-4EDD-BB85-2C7EC7DB332D


In [7]:
build_df = spy.assets.build(Region, metadata_df)
build_df

,Build Path,Build Asset,Build Template,Build Result
0,Cooling Tower Health,North America,Region,Success


,Name,Type,Asset,Path,Asset Object,Template,Build Result,Workbook Type,Object
0,A,Asset,A,Cooling Tower Health >> North America >> 1,Cooling Tower Health >> North America >> 1 >> A,Area,Success,NaN,NaN
1,B,Asset,B,Cooling Tower Health >> North America >> 1,Cooling Tower Health >> North America >> 1 >> B,Area,Success,NaN,NaN
2,C,Asset,C,Cooling Tower Health >> North America >> 1,Cooling Tower Health >> North America >> 1 >> C,Area,Success,NaN,NaN
3,G,Asset,G,Cooling Tower Health >> North America >> 1,Cooling Tower Health >> North America >> 1 >> G,Area,Success,NaN,NaN
4,H,Asset,H,Cooling Tower Health >> North America >> 1,Cooling Tower Health >> North America >> 1 >> H,Area,Success,NaN,NaN
5,I,Asset,I,Cooling Tower Health >> North America >> 1,Cooling Tower Health >> North America >> 1 >> I,Area,Success,NaN,NaN
6,J,Asset,J,Cooling Tower Health >> North America >> 1,Cooling Tower Health >> North America >> 1 >> J,Area,Success,NaN,NaN
7,K,Asset,K,Cooling Tower Health >> North America >> 1,Cooling Tower Health >> North America >> 1 >> K,Area,Success,NaN,NaN
8,1,Asset,1,Cooling Tower Health >> North America,Cooling Tower Health >> North America >> 1,Cooling Tower,Success,NaN,NaN
9,D,Asset,D,Cooling Tower Health >> North America >> 2,Cooling Tower Health >> North America >> 2 >> D,Area,Success,NaN,NaN


In [8]:
spy.push(metadata=build_df, workbook='SPy Documentation Examples >> Cooling Tower Health')

,Name,Type,Asset,Path,Asset Object,Template,Build Result,Workbook Type,Object,Datasource Class,Datasource ID,Data ID,ID,Push Result
0,A,Asset,A,Cooling Tower Health >> North America >> 1,Cooling Tower Health >> North America >> 1 >> A,Area,Success,NaN,NaN,Seeq Data Lab,Seeq Data Lab,[D168F6D6-C710-4413-9E4C-3C5A66B0E6F6] {Asset}...,1F777295-6313-43C8-979D-A9D662226DBD,Success
1,B,Asset,B,Cooling Tower Health >> North America >> 1,Cooling Tower Health >> North America >> 1 >> B,Area,Success,NaN,NaN,Seeq Data Lab,Seeq Data Lab,[D168F6D6-C710-4413-9E4C-3C5A66B0E6F6] {Asset}...,3249ECE8-43A8-44C8-8E0C-E8B093D54BDA,Success
2,C,Asset,C,Cooling Tower Health >> North America >> 1,Cooling Tower Health >> North America >> 1 >> C,Area,Success,NaN,NaN,Seeq Data Lab,Seeq Data Lab,[D168F6D6-C710-4413-9E4C-3C5A66B0E6F6] {Asset}...,76D65E4D-32DB-45BC-95AD-0BCBB16A293C,Success
3,G,Asset,G,Cooling Tower Health >> North America >> 1,Cooling Tower Health >> North America >> 1 >> G,Area,Success,NaN,NaN,Seeq Data Lab,Seeq Data Lab,[D168F6D6-C710-4413-9E4C-3C5A66B0E6F6] {Asset}...,92E6AD10-7997-43C7-9503-E8FAC5C3C310,Success
4,H,Asset,H,Cooling Tower Health >> North America >> 1,Cooling Tower Health >> North America >> 1 >> H,Area,Success,NaN,NaN,Seeq Data Lab,Seeq Data Lab,[D168F6D6-C710-4413-9E4C-3C5A66B0E6F6] {Asset}...,A3ADDDF3-80BD-45C0-B484-175AB0581DF5,Success
5,I,Asset,I,Cooling Tower Health >> North America >> 1,Cooling Tower Health >> North America >> 1 >> I,Area,Success,NaN,NaN,Seeq Data Lab,Seeq Data Lab,[D168F6D6-C710-4413-9E4C-3C5A66B0E6F6] {Asset}...,0289173F-0AED-447E-893C-B4A947AED62B,Success
6,J,Asset,J,Cooling Tower Health >> North America >> 1,Cooling Tower Health >> North America >> 1 >> J,Area,Success,NaN,NaN,Seeq Data Lab,Seeq Data Lab,[D168F6D6-C710-4413-9E4C-3C5A66B0E6F6] {Asset}...,9879E756-69EC-42B5-B5B1-4E988BADB8C8,Success
7,K,Asset,K,Cooling Tower Health >> North America >> 1,Cooling Tower Health >> North America >> 1 >> K,Area,Success,NaN,NaN,Seeq Data Lab,Seeq Data Lab,[D168F6D6-C710-4413-9E4C-3C5A66B0E6F6] {Asset}...,3C3163F5-5A72-4C1E-B403-CAB2796E26C8,Success
8,1,Asset,1,Cooling Tower Health >> North America,Cooling Tower Health >> North America >> 1,Cooling Tower,Success,NaN,NaN,Seeq Data Lab,Seeq Data Lab,[D168F6D6-C710-4413-9E4C-3C5A66B0E6F6] {Asset}...,7D7D7A09-1340-41E1-AABA-8D6D22DB156A,Success
9,D,Asset,D,Cooling Tower Health >> North America >> 2,Cooling Tower Health >> North America >> 2 >> D,Area,Success,NaN,NaN,Seeq Data Lab,Seeq Data Lab,[D168F6D6-C710-4413-9E4C-3C5A66B0E6F6] {Asset}...,39AC5CBD-82D8-43F4-B108-8953957F2D88,Success
